In [56]:
import os, glob
import pandas as pd
import numpy as np
import re

In [80]:
def load_and_preprocess(f):
    classif = re.match('tickers_(\w+).csv', f).group(1)
    df = pd.read_csv(f)
    df = df.iloc[:,:-1]
    df['classif'] = classif
    return df

In [81]:
dfs = [load_and_preprocess(f) for f in glob.glob('tickers_*.csv')]

df = pd.concat(dfs, ignore_index=True)

In [85]:
df_class = df.reset_index().pivot_table('index',['Symbol','Name'],'classif', aggfunc=lambda x: len(x) > 0).fillna(False)

In [86]:
df_class.tech.sum()

625

In [104]:
df_class = df.reset_index().pivot_table('index',['Symbol','Name'],'classif', aggfunc=lambda x: len(x) > 0).fillna(False)
df_class.columns.name = None
columns = df_class.columns
df_class = df_class[sorted([c for c in columns if 'cap' not in c])+sorted([c for c in columns if 'cap' in c])]
df_class.reset_index(inplace=True)

df_meta = df.groupby('Symbol')[['LastSale','MarketCap','IPOyear']].mean().join(
df.groupby('Symbol')[['Sector','Industry']].last()
).reset_index()

df_merged = df_meta.merge(df_class, on='Symbol', how='outer')

df_merged.to_csv('tickers.csv', index=False)

In [105]:
df_merged = pd.read_csv('tickers.csv')

In [106]:
df_merged.iloc[:,7:].sum()

basicindustries         317
consumergoods           148
consumernondurables     238
consumerservices        825
energy                  314
finance                1054
healthcare              859
misc                    156
publicutilities         271
tech                    625
transportation          123
capitalgoods            371
largecap                706
megacap                  26
microcap               1335
midcap                 1184
nanocap                1182
smallcap               1840
dtype: int64

In [107]:
df_merged.dtypes

Symbol                  object
LastSale               float64
MarketCap              float64
IPOyear                float64
Sector                  object
Industry                object
Name                    object
basicindustries           bool
consumergoods             bool
consumernondurables       bool
consumerservices          bool
energy                    bool
finance                   bool
healthcare                bool
misc                      bool
publicutilities           bool
tech                      bool
transportation            bool
capitalgoods              bool
largecap                  bool
megacap                   bool
microcap                  bool
midcap                    bool
nanocap                   bool
smallcap                  bool
dtype: object